In [2]:
# Load libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

<h2>Klassifizierung</h2>
<h4>Hier nur die endgültige Klassifizierung, auskommentiert die Trainings-Klassifizierung</h4>
<h5>Vorgehen:  </h5>

In [3]:
dataClaNEG = pd.read_csv("C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\preprocessedDataNEG.csv", sep=',', usecols = ['Text', 'Label'], engine='python')
dataClaPOS = pd.read_csv("C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\preprocessedDataPos.csv", sep=',', usecols = ['Text', 'Label'], engine='python')
dataClaTEST = pd.read_csv("C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\preprocessedDataTest.csv", sep=',', usecols = ['Text', 'Label'], engine='python')

tupelNeg = dataClaNEG.shape
lengthNeg = tupelNeg[0]
tupelPos = dataClaPOS.shape #kürzer als 1500 weil zwei Einträge im Orginalen fehlen!
lengthPos = tupelPos[0]

lengthTrain = lengthNeg + lengthPos

print(lengthNeg)
print(lengthPos)
print((dataClaTEST.shape)[0]) #1500 1498 1000

1500
1498
1000


In [4]:
dataCla = pd.concat([dataClaPOS, dataClaNEG, dataClaTEST], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
print('Shape all data: ', dataCla.shape) #müsste 3998, 2

Shape all data:  (3998, 2)


<h2>TFIDF</h2>

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

dataVEC = pd.DataFrame((vectorizer.fit_transform(dataCla['Text']).toarray()), columns = vectorizer.get_feature_names())

<h4>Labels wieder einfügen</h4>

In [6]:
dataClaLabel = pd.concat([dataVEC, dataCla], axis=1, join='outer', join_axes=[dataVEC.index], ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True, sort=False)

dataClaLabel.drop(['Text'], axis=1, inplace = True)

<h4>Garbage Collector</h4>

In [7]:
dataVEC, dataCla, dataClaNEG, dataClaPOS, dataClaTEST = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
lst = [dataVEC, dataCla, dataClaNEG, dataClaPOS, dataClaTEST]
del dataVEC, dataCla, dataClaNEG, dataClaPOS, dataClaTEST
del lst

In [8]:
array = dataClaLabel.values

tupelMes = dataClaLabel.shape

lengthLabel = tupelMes[0]
widthLabel = tupelMes[1]

train_x = array[0:lengthTrain,0:(widthLabel-1)]
train_y = array[0:lengthTrain,widthLabel-1]
test_i = array[lengthTrain:lengthLabel, 0:(widthLabel-1)]

<h4>Im folgenden wird getestet welches Modell am besten ist für gegebene Trainingsdaten.
Auskommentiert</h4>

In [9]:
#validation_size = 0.20
#seed = 42
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)
seed = 42 #7
scoring = 'accuracy'

models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, train_x, train_y, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)


<h4>Ergebnisse:</h4>
<table>
  <tr>
    <td>
      LogisticRegression
    </td>
    <td>
      0,85
    </td>
  </tr>
  <tr>
    <td>
      KNeighborsClassifier
    </td>
    <td>
      0,75
    </td>
  </tr>
  <tr>
    <td>
      DecisionTreeClassifier
    </td>
    <td>
      0,69
    </td>
  </tr>
  <tr>
    <td>
      GaussianNB
    </td>
    <td>
      0,61
    </td>
  </tr>
  <tr>
    <td>
      SVM
    </td>
    <td>
      0,51
    </td>
  </tr>
</table>

<h2>Predictions mit LogisticRegression</h2>

In [10]:
LR = LogisticRegression()
LR.fit(train_x, train_y)
predictions = LR.predict(test_i)
predictions = predictions.astype(int)
print(predictions)
print(predictions[0])
print(len(predictions))
import numpy
numpy.savetxt('Predictions.csv', predictions.astype(int), fmt='%i', delimiter=',')

c:\users\ojkbe\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1
 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0
 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1
 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0
 1 0 0 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0
 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0
 1 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 1 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1
 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1
 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0
 1 0 0 1 1 0 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0
 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1
 1 1 1 1 0 0 0 0 1 0 0 0 